<H1>Analysis for the approach of a match<H1>

In [ ]:
#Imports
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import math
from matplotlib.ticker import MaxNLocator
import matplotlib.patches as patches
from matplotlib import animation
from IPython.display import HTML
import warnings
warnings.filterwarnings('ignore')

In [ ]:
#Data reader
games=pd.read_csv("../input/nfl-big-data-bowl-2022/games.csv",parse_dates=[3,4])  
PFFScoutingData=pd.read_csv('../input/nfl-big-data-bowl-2022/PFFScoutingData.csv')  
players=pd.read_csv('../input/nfl-big-data-bowl-2022/players.csv')  
plays=pd.read_csv("../input/nfl-big-data-bowl-2022/plays.csv")  
tracking2020=pd.read_csv('../input/nfl-big-data-bowl-2022/tracking2020.csv')  

<H3>Select your rival<H3>

In [ ]:
rival='ATL'

In [ ]:
#We calculate the final result including the points of the play
plays = pd.merge(games, plays, on="gameId", how="inner")
conditions = [
    (plays["specialTeamsResult"]=="Kick Attempt Good") & (plays["specialTeamsPlayType"]=="Extra Point") & (plays["possessionTeam"]==plays["homeTeamAbbr"]),
    (plays["specialTeamsResult"]=="Kick Attempt Good") & (plays["specialTeamsPlayType"]=="Field Goal") & (plays["possessionTeam"]==plays["homeTeamAbbr"]),
    (plays["specialTeamsResult"]=="Kick Attempt Good") & (plays["specialTeamsPlayType"]=="Extra Point") & (plays["possessionTeam"]!=plays["homeTeamAbbr"]),
    (plays["specialTeamsResult"]=="Kick Attempt Good") & (plays["specialTeamsPlayType"]=="Field Goal") & (plays["possessionTeam"]!=plays["homeTeamAbbr"]),
    True
    ]

values = [1,3,-1,-3,0]

plays["points"] = np.select(conditions, values)

plays["preSnapHomeScore"] = plays[["preSnapHomeScore","points"]].sum(axis=1).where(plays["points"] > 0, plays["preSnapHomeScore"])
plays["points"]= plays["points"]*-1
plays["preSnapVisitorScore"] = plays[["preSnapVisitorScore","points"]].sum(axis=1).where(plays["points"] > 0, plays["preSnapVisitorScore"])

In [ ]:
#We select the opponent's games and fields of interest
infoGames = plays[ (plays["homeTeamAbbr"] == rival) | (plays["visitorTeamAbbr"]== rival)]

<H2>1. General information of defeats and victories of the opponent<H2>

In [ ]:
#We select the last play to keep the las result
idx1 = infoGames.groupby(["gameId"])["playId"].transform(max) == infoGames["playId"]
finalResult = infoGames[idx1]

#We create the field "home/visitor"
finalResult["HomeVisitor"] = ["Home" if club==rival else "Visitor" for club in finalResult["homeTeamAbbr"]]

#We check whether you have won or lost.
def victory(row):
    if row["HomeVisitor"]=="Visitor" and row["preSnapHomeScore"]>row["preSnapVisitorScore"]:
        return "Defeat" 
    elif row["HomeVisitor"]=="Visitor" and row["preSnapHomeScore"]<row["preSnapVisitorScore"]:
        return "Victory" 
    elif row["HomeVisitor"]=="Home" and row["preSnapHomeScore"]>row["preSnapVisitorScore"]:
        return "Victory"
    elif row["HomeVisitor"]=="Home" and row["preSnapHomeScore"]<row["preSnapVisitorScore"]:
        return "Defeat"
    else:
        return "Draw" 
finalResult["Result"] = finalResult.apply(victory, axis=1)


#We select the fields we need for the chart
finalResult = finalResult[["season","HomeVisitor","Result"]]

#We add a column with 1 to calculate the total of wins, losses and draws
finalResult['total'] = 1

#We calculate the total of victories, defeats and draws grouping by season, HomeVisitor, Result
finalResult['Sum'] = finalResult['total'].groupby([finalResult['season'],finalResult['HomeVisitor'],finalResult['Result']]).transform('sum')

#We delete the duplicate rows and select the ones we need to make the table
finalResult = finalResult.drop_duplicates()
finalResult = finalResult[["season","HomeVisitor","Result","Sum"]]

#We draw the table modifying the labels
plot1 = sns.catplot(kind='bar', data=finalResult, col='season', x='HomeVisitor', y='Sum', hue='Result', estimator=sum, ci=None,
            order=sorted(finalResult.HomeVisitor.unique()), col_order=sorted(finalResult.season.unique()))
plot1.set_xlabels('Home vs Visitor', fontsize=15) # not set_label
plot1.set_ylabels('Frequency', fontsize=15)
plt.show()

<H2>2. Type of kickoff in the last season<H2>

In [ ]:
Games = infoGames[["gameId","season","playId","possessionTeam"]]
Games2020 = Games[(Games["possessionTeam"]==rival) & (Games["season"]==2020)]
PFFSD_filtered = PFFScoutingData[["gameId","playId","kickType"]]
Games_TypeKick = pd.merge(Games2020, PFFSD_filtered, on=["gameId","playId"], how="inner")
kick= Games_TypeKick.groupby(["kickType"]).size()
sizes = ((kick / kick.sum()) * 100).to_list()
labels = kick.index.to_list()
labels = [f'{l}, {s:0.1f}%' for l, s in zip(labels, sizes)]
plot2 = plt.pie(kick)
plt.legend(bbox_to_anchor=(0.9, 0.9), loc='upper left', labels=labels)
plt.show()

<H2>3. Games played by each player in the last season. See which player is more important in the team.<H2>

In [ ]:
GamesPlayed=infoGames[["gameId","homeTeamAbbr"]]
GamesPlayed["team"]=np.where(GamesPlayed['homeTeamAbbr']== rival, "home", "visitor")
GamesPlayed=GamesPlayed.drop_duplicates()
tracking2020_aux=tracking2020[["gameId","nflId","team"]]
tracking2020_aux=tracking2020_aux.groupby(["gameId","nflId","team"]).count().reset_index()
InfoPlayers = pd.merge(GamesPlayed, tracking2020_aux, on=["gameId","team"], how="inner")
playersName=players[["nflId","Position","displayName"]]
playersInfoName=InfoPlayers.join(playersName.set_index('nflId'), on='nflId')
playersInfoName=playersInfoName[["displayName","Position"]]
playersInfoName["count"]=1
playersInfoName=playersInfoName.groupby(["displayName","Position"]).sum().reset_index().sort_values(by=["count"],ascending=False)
playersInfoName = playersInfoName.set_axis(['Player Name', 'Position', 'Games Played'], axis=1)
print (playersInfoName.to_string(index=False))

<H2>4. Frequency of the attacking side of the last game. Where they attack the most.<H2>

In [ ]:
gamesPos = Games[(Games["possessionTeam"]==rival) & (Games["gameId"]==Games.gameId.max())]
PFFScoutingData_aux = PFFScoutingData[["gameId","playId","kickDirectionActual"]]
gamesPos_PFFSD = pd.merge(gamesPos, PFFScoutingData_aux, on=["gameId","playId"], how="inner")
directionGames = gamesPos_PFFSD["kickDirectionActual"].value_counts(dropna=True)
plot3=directionGames.plot.bar(color=["orange","blue","green"])
plot3.yaxis.set_major_locator(MaxNLocator(integer=True))
plt.show()

<H2>5. Team that has played in the previous game<H2>

In [ ]:
lastGame=infoGames["gameId"].max()
LastGamePlay=games[games["gameId"]==lastGame]
HomeAway="home"
if LastGamePlay.homeTeamAbbr.to_string()==rival:
    HomeAway= "home"
else: 
    HomeAway= "away"
lastGameInfo=tracking2020[(tracking2020["gameId"]==lastGame) & (tracking2020["team"]==HomeAway)]
playersLastGame= lastGameInfo["nflId"].unique()
ListPlayers=players[players.nflId.isin(playersLastGame)]
ListPlayers=ListPlayers.drop(["nflId"],axis=1).sort_values("Position")
ListPlayers["age"]= (pd.Timestamp('now')-pd.to_datetime(ListPlayers["birthDate"], format='%Y-%m-%d')).astype('<m8[Y]')
ListPlayers=ListPlayers.drop(["birthDate"],axis=1)
print(ListPlayers.to_string(index=False))

<H2>6. Age distribution of the players from the previous game<H2>

In [ ]:
axAge = sns.distplot(ListPlayers["age"])
axAge.set(xticks=list(range(18,38)))
axAge.set_title('Players age Distribution')
plt.show()

<H2>7. Weight distribution of the players from the previous game<H2>

In [ ]:
axWeight = sns.distplot(ListPlayers["weight"])
axWeight.set_title('Players weight Distribution')
plt.show()

<H2>8. Graph of the points scored for and against in each quarter. See which times he suffers the most and where he is strongest<H2>

In [ ]:
#We select the matches of the last season with the fields of interest (We reuse the infoGames variable)
pointsQuarter = infoGames[infoGames["season"]==2020]
pointsQuarter = pointsQuarter[["gameId","playId","quarter","homeTeamAbbr","visitorTeamAbbr","preSnapHomeScore","preSnapVisitorScore"]]

#We filter to keep the score at the end of every quarter
idx2 = pointsQuarter.groupby(["gameId","quarter"])["playId"].transform(max) == pointsQuarter["playId"]
pointsQuarter = pointsQuarter[idx2]

pointsHome = pointsQuarter[["gameId","quarter","homeTeamAbbr","preSnapHomeScore"]]
pointsHome = pointsHome.rename({'homeTeamAbbr': 'team', 'preSnapHomeScore': 'score'}, axis=1) 

pointsVisitor = pointsQuarter[["gameId","quarter","visitorTeamAbbr","preSnapVisitorScore"]]
pointsVisitor = pointsQuarter.rename({'visitorTeamAbbr': 'team', 'preSnapVisitorScore': 'score'}, axis=1) 

pointsGame = pd.concat([pointsHome, pointsVisitor]).reset_index(drop=True)
pointsGame = pointsGame.rename(columns={'gameId':'Date'})
width=int(len(pointsGame["Date"].drop_duplicates())/4)

plot4= sns.relplot(
    data=pointsGame, x="quarter", y="score",
    col="Date", hue="team", style="team", col_wrap=width,
    kind="line",
)
plot4.set( xlabel = "Quarter", ylabel = "Score", xticks=([1,2,3,4]))
plot4._legend.set_title("Team")
plt.show()

<H2>9. Match to analyze<H2>

<H3>Select the match<H3>

In [ ]:
game=2021010310

In [ ]:
tracking2020_aux=tracking2020[tracking2020["gameId"]==game]

<H3>Game play log<H3>

In [ ]:
listGames=plays[plays["gameId"]==game]
listGames=listGames[["playId","quarter","possessionTeam","specialTeamsPlayType","specialTeamsResult"]]
print(listGames.to_string(index=False))

This function is taken from the post created by JARON_MICHAL. https://www.kaggle.com/jaronmichal/tracking-data-visualization

In [ ]:
# Change size of the figure
plt.rcParams['figure.figsize'] = [12, 8]
def drawPitch(width, height, color="w"):
    fig = plt.figure()
    ax = plt.axes(xlim=(-10, width + 30), ylim=(-15, height + 5))
    plt.axis('off')

    # Grass around pitch
    rect = patches.Rectangle((-10, -5), width + 40, height + 10, linewidth=1, facecolor='#3f995b', capstyle='round')
    ax.add_patch(rect)
    ###################

    # Pitch boundaries
    rect = plt.Rectangle((0, 0), width + 20, height, ec=color, fc="None", lw=2)
    ax.add_patch(rect)
    ###################

    # vertical lines - every 5 yards
    for i in range(21):
        plt.plot([10 + 5 * i, 10 + 5 * i], [0, height], c="w", lw=2)
    ###################
        
    # distance markers - every 10 yards
    for yards in range(10, width, 10):
        yards_text = yards if yards <= width / 2 else width - yards
        # top markers
        plt.text(10 + yards - 2, height - 7.5, yards_text, size=15, c="w", weight="bold")
        # botoom markers
        plt.text(10 + yards - 2, 7.5, yards_text, size=15, c="w", weight="bold", rotation=180)
    ###################

    # yards markers - every yard
    # bottom markers
    for x in range(20):
        for j in range(1, 5):
            plt.plot([10 + x * 5 + j, 10 + x * 5 + j], [1, 3], color="w", lw=2)

    # top markers
    for x in range(20):
        for j in range(1, 5):
            plt.plot([10 + x * 5 + j, 10 + x * 5 + j], [height - 1, height - 3], color="w", lw=2)

    # middle bottom markers
    y = (height - 18.5) / 2
    for x in range(20):
        for j in range(1, 5):
            plt.plot([10 + x * 5 + j, 10 + x * 5 + j], [y, y + 2], color="w", lw=2)

    # middle top markers
    for x in range(20):
        for j in range(1, 5):
            plt.plot([10 + x * 5 + j, 10 + x * 5 + j], [height - y, height - y - 2], color="w", lw=2)
    ###################

    # draw home end zone
    plt.text(2.5, (height - 15) / 2, "HOME", size=30, c="w", weight="bold", rotation=90)
    rect = plt.Rectangle((0, 0), 10, height, ec=color, fc="#0064dc", lw=2)
    ax.add_patch(rect)

    # draw away end zone    
    plt.text(111, (height - 15) / 2, "AWAY", size=30, c="w", weight="bold", rotation=-90)
    rect = plt.Rectangle((width + 10, 0), 10, height, ec=color, fc="#c80014", lw=2)
    ax.add_patch(rect)
    ###################
    
    # draw extra spot point
    # left
    y = (height - 3) / 2
    plt.plot([10 + 2, 10 + 2], [y, y + 3], c="w", lw=2)
    
    # right
    plt.plot([width + 10 - 2, width + 10 - 2], [y, y + 3], c="w", lw=2)
    ###################
    
    # draw goalpost
    goal_width = 6 # yards
    y = (height - goal_width) / 2
    # left
    plt.plot([0, 0], [y, y + goal_width], "-", c="y", lw=10, ms=20)
    # right
    plt.plot([width + 20, width + 20], [y, y + goal_width], "-", c="y", lw=10, ms=20)
    
    return fig, ax

In [ ]:
def extract_one_game(game_id, play_id, df):
    game = df[(df.gameId == game_id) & (df.playId == play_id)]
    home = {}
    away = {}
    balls = []
    
    players = game.sort_values(['frameId'], ascending=True).groupby('nflId')
    for id, dx in players:
        jerseyNumber = int(dx.jerseyNumber.iloc[0])
        if dx.team.iloc[0] == "home":
            home[jerseyNumber] = list(zip(dx.x.tolist(), dx.y.tolist()))
        elif dx.team.iloc[0] == "away":
            away[jerseyNumber] = list(zip(dx.x.tolist(), dx.y.tolist()))


    ball_df = game.sort_values(['frameId'], ascending=True) 
    ball_df = ball_df[ball_df.team == "football"]
    balls = list(zip(ball_df.x.tolist(), ball_df.y.tolist()))
    return home, away, balls



In [ ]:
def animate_one_play(game_id, play_id, df):
    fig, ax = drawPitch(100, 53.3)
    
    home, away, balls = extract_one_game(game_id, play_id, df)

    team_left, = ax.plot([], [], 'o', markersize=20, markerfacecolor="r", markeredgewidth=2, markeredgecolor="white", zorder=7)
    team_right, = ax.plot([], [], 'o', markersize=20, markerfacecolor="b", markeredgewidth=2, markeredgecolor="white", zorder=7)
    ball, = ax.plot([], [], 'o', markersize=10, markerfacecolor="black", markeredgewidth=2, markeredgecolor="white", zorder=7)
    drawings = [team_left, team_right, ball]

    def init():
        team_left.set_data([], [])
        team_right.set_data([], [])
        ball.set_data([], [])
        return drawings

    def draw_teams(i):
        X = []
        Y = []
        for k, v in home.items():
            x, y = v[i]
            X.append(x)
            Y.append(y)
        team_left.set_data(X, Y)
        
        X = []
        Y = []
        for k, v in away.items():
            x, y = v[i]
            X.append(x)
            Y.append(y)
        team_right.set_data(X, Y)

    def animate(i):
        draw_teams(i)
        
        x, y = balls[i]
        ball.set_data([x, y])
        return drawings
    
    # !May take a while!
    anim = animation.FuncAnimation(fig, animate, init_func=init,
                                   frames=len(balls), interval=100, blit=True)

    return HTML(anim.to_html5_video())

<H3>Select the play<H3>

In [ ]:
play=40

In [ ]:
animate_one_play(game, play, tracking2020_aux)